# Examples and Exercises from Think Stats, 2nd Edition

http://thinkstats2.com

Copyright 2016 Allen B. Downey

MIT License: https://opensource.org/licenses/MIT


In [ ]:
from __future__ import print_function, division

%matplotlib inline

import numpy as np

import random

import thinkstats2
import thinkplot

## Least squares

One more time, let's load up the NSFG data.

In [ ]:
import first
live, firsts, others = first.MakeFrames()
live = live.dropna(subset=['agepreg', 'totalwgt_lb'])
ages = live.agepreg
weights = live.totalwgt_lb

The following function computes the intercept and slope of the least squares fit.

In [ ]:
from thinkstats2 import Mean, MeanVar, Var, Std, Cov

def LeastSquares(xs, ys):
    meanx, varx = MeanVar(xs)
    meany = Mean(ys)

    slope = Cov(xs, ys, meanx, meany) / varx
    inter = meany - slope * meanx

    return inter, slope

Here's the least squares fit to birth weight as a function of mother's age.

In [ ]:
inter, slope = LeastSquares(ages, weights)
inter, slope

The intercept is often easier to interpret if we evaluate it at the mean of the independent variable.

In [ ]:
inter + slope * 25

And the slope is easier to interpret if we express it in pounds per decade (or ounces per year).

In [ ]:
slope * 10

The following function evaluates the fitted line at the given `xs`.

In [ ]:
def FitLine(xs, inter, slope):
    fit_xs = np.sort(xs)
    fit_ys = inter + slope * fit_xs
    return fit_xs, fit_ys

And here's an example.

In [ ]:
fit_xs, fit_ys = FitLine(ages, inter, slope)

Here's a scatterplot of the data with the fitted line.

In [ ]:
# this turned out pretty nice I felt that the line that it found seemed reasonable and would represent the data well.
thinkplot.Scatter(ages, weights, color='blue', alpha=0.1, s=10)
thinkplot.Plot(fit_xs, fit_ys, color='white', linewidth=3)
thinkplot.Plot(fit_xs, fit_ys, color='red', linewidth=2)
thinkplot.Config(xlabel="Mother's age (years)",
                 ylabel='Birth weight (lbs)',
                 axis=[10, 45, 0, 15],
                 legend=False)

## Residuals

The following functon computes the residuals.

In [ ]:
def Residuals(xs, ys, inter, slope):
    xs = np.asarray(xs)
    ys = np.asarray(ys)
    res = ys - (inter + slope * xs)
    return res

Now we can add the residuals as a column in the DataFrame.

In [ ]:
live['residual'] = Residuals(ages, weights, inter, slope)

To visualize the residuals, I'll split the respondents into groups by age, then plot the percentiles of the residuals versus the average age in each group.

First I'll make the groups and compute the average age in each group.

In [ ]:
bins = np.arange(10, 48, 3)
indices = np.digitize(live.agepreg, bins)
groups = live.groupby(indices)

age_means = [group.agepreg.mean() for _, group in groups][1:-1]
age_means

Next I'll compute the CDF of the residuals in each group.

In [ ]:
cdfs = [thinkstats2.Cdf(group.residual) for _, group in groups][1:-1]

The following function plots percentiles of the residuals against the average age in each group.

In [ ]:
def PlotPercentiles(age_means, cdfs):
    thinkplot.PrePlot(3)
    for percent in [75, 50, 25]:
        weight_percentiles = [cdf.Percentile(percent) for cdf in cdfs]
        label = '%dth' % percent
        thinkplot.Plot(age_means, weight_percentiles, label=label)

The following figure shows the 25th, 50th, and 75th percentiles.

Curvature in the residuals suggests a non-linear relationship.

In [ ]:
PlotPercentiles(age_means, cdfs)

thinkplot.Config(xlabel="Mother's age (years)",
                 ylabel='Residual (lbs)',
                 xlim=[10, 45])

## Sampling distribution

To estimate the sampling distribution of `inter` and `slope`, I'll use resampling.

In [ ]:
def SampleRows(df, nrows, replace=False):
    """Choose a sample of rows from a DataFrame.

    df: DataFrame
    nrows: number of rows
    replace: whether to sample with replacement

    returns: DataDf
    """
    indices = np.random.choice(df.index, nrows, replace=replace)
    sample = df.loc[indices]
    return sample

def ResampleRows(df):
    """Resamples rows from a DataFrame.

    df: DataFrame

    returns: DataFrame
    """
    return SampleRows(df, len(df), replace=True)

The following function resamples the given dataframe and returns lists of estimates for `inter` and `slope`.

In [ ]:
def SamplingDistributions(live, iters=101):
    t = []
    for _ in range(iters):
        sample = ResampleRows(live)
        ages = sample.agepreg
        weights = sample.totalwgt_lb
        estimates = LeastSquares(ages, weights)
        t.append(estimates)

    inters, slopes = zip(*t)
    return inters, slopes

Here's an example.

In [ ]:
inters, slopes = SamplingDistributions(live, iters=1001)

The following function takes a list of estimates and prints the mean, standard error, and 90% confidence interval.

In [ ]:
def Summarize(estimates, actual=None):
    mean = Mean(estimates)
    stderr = Std(estimates, mu=actual)
    cdf = thinkstats2.Cdf(estimates)
    ci = cdf.ConfidenceInterval(90)
    print('mean, SE, CI', mean, stderr, ci)

Here's  the summary for `inter`.

In [ ]:
Summarize(inters)

And for `slope`.

In [ ]:
Summarize(slopes)

**Exercise:** Use `ResampleRows` and generate a list of estimates for the mean birth weight.  Use `Summarize` to compute the SE and CI for these estimates.

In [ ]:
## So one of the main ideas in the central limit theorem is that if we take many samples.
## Eventually we will be able to determine important things about the population.
## Here we are only doing 1000 itterations but we are looking for the mean over and overagain.
## This gives us insight into what the population mean actually is without having all of the information
## about the population itself.
iters = 1000
estimates = [ResampleRows(live).totalwgt_lb.mean()
             for _ in range(iters)]
Summarize(estimates)
# Solution goes here

## Visualizing uncertainty

To show the uncertainty of the estimated slope and intercept, we can generate a fitted line for each resampled estimate and plot them on top of each other.

In [ ]:
## For this one we are taking our samples that we took and building lines based on the samples mean and y intercept.
## The problem is there are an infinite number of possibilities for how we would draw the line of best fit.
## If we choose poorly then we wont make accurate predictions when we try to interpolate data within the domain of x or
## extrapolate data outside of the sample domain of x.
## The goal is to accurately predict the values y.
## By taking many iterations we can find averages based on what we learned from manny samples.
for slope, inter in zip(slopes, inters):
    fxs, fys = FitLine(age_means, inter, slope)
    thinkplot.Plot(fxs, fys, color='gray', alpha=0.01)
    
thinkplot.Config(xlabel="Mother's age (years)",
                 ylabel='Residual (lbs)',
                 xlim=[10, 45])

Or we can make a neater (and more efficient plot) by computing fitted lines and finding percentiles of the fits for each value of the dependent variable.

In [ ]:
def PlotConfidenceIntervals(xs, inters, slopes, percent=90, **options):
    fys_seq = []
    for inter, slope in zip(inters, slopes):
        fxs, fys = FitLine(xs, inter, slope)
        fys_seq.append(fys)

    p = (100 - percent) / 2
    percents = p, 100 - p
    low, high = thinkstats2.PercentileRows(fys_seq, percents)
    thinkplot.FillBetween(fxs, low, high, **options)

This example shows the confidence interval for the fitted values at each mother's age.

In [ ]:
PlotConfidenceIntervals(age_means, inters, slopes, percent=90, 
                        color='gray', alpha=0.3, label='90% CI')
PlotConfidenceIntervals(age_means, inters, slopes, percent=50,
                        color='gray', alpha=0.5, label='50% CI')

thinkplot.Config(xlabel="Mother's age (years)",
                 ylabel='Residual (lbs)',
                 xlim=[10, 45])

## Coefficient of determination



The coefficient compares the variance of the residuals to the variance of the dependent variable.

In [ ]:
def CoefDetermination(ys, res):
    return 1 - Var(res) / Var(ys)

For birth weight and mother's age $R^2$ is very small, indicating that the mother's age predicts a small part of the variance in birth weight.

In [ ]:
inter, slope = LeastSquares(ages, weights)
res = Residuals(ages, weights, inter, slope)
r2 = CoefDetermination(weights, res)
r2

We can confirm that $R^2 = \rho^2$:

In [ ]:
print('rho', thinkstats2.Corr(ages, weights))
print('R', np.sqrt(r2))    

To express predictive power, I think it's useful to compare the standard deviation of the residuals to the standard deviation of the dependent variable, as a measure RMSE if you try to guess birth weight with and without taking into account mother's age.

In [ ]:
print('Std(ys)', Std(weights))
print('Std(res)', Std(res))

As another example of the same idea, here's how much we can improve guesses about IQ if we know someone's SAT scores.

In [ ]:
var_ys = 15**2
rho = 0.72
r2 = rho**2
var_res = (1 - r2) * var_ys
std_res = np.sqrt(var_res)
std_res

## Hypothesis testing with slopes

Here's a `HypothesisTest` that uses permutation to test whether the observed slope is statistically significant.

In [ ]:
class SlopeTest(thinkstats2.HypothesisTest):

    def TestStatistic(self, data):
        ages, weights = data
        _, slope = thinkstats2.LeastSquares(ages, weights)
        return slope

    def MakeModel(self):
        _, weights = self.data
        self.ybar = weights.mean()
        self.res = weights - self.ybar

    def RunModel(self):
        ages, _ = self.data
        weights = self.ybar + np.random.permutation(self.res)
        return ages, weights

And it is.

In [ ]:
ht = SlopeTest((ages, weights))
pvalue = ht.PValue()
pvalue

Under the null hypothesis, the largest slope we observe after 1000 tries is substantially less than the observed value.

In [ ]:
ht.actual, ht.MaxTestStat()

We can also use resampling to estimate the sampling distribution of the slope.

In [ ]:
sampling_cdf = thinkstats2.Cdf(slopes)

The distribution of slopes under the null hypothesis, and the sampling distribution of the slope under resampling, have the same shape, but one has mean at 0 and the other has mean at the observed slope.

To compute a p-value, we can count how often the estimated slope under the null hypothesis exceeds the observed slope, or how often the estimated slope under resampling falls below 0.

In [ ]:
# there is a clear difference in means here even though they are small.
# This seems to coincide with the fact that the confidence intervals did not overlap.
thinkplot.PrePlot(2)
thinkplot.Plot([0, 0], [0, 1], color='0.8')
ht.PlotCdf(label='null hypothesis')

thinkplot.Cdf(sampling_cdf, label='sampling distribution')

thinkplot.Config(xlabel='slope (lbs / year)',
                   ylabel='CDF',
                   xlim=[-0.03, 0.03],
                   legend=True, loc='upper left')

Here's how to get a p-value from the sampling distribution.

In [ ]:
# Pvalue of zero is significant.
pvalue = sampling_cdf[0]
pvalue

## Resampling with weights

Resampling provides a convenient way to take into account the sampling weights associated with respondents in a stratified survey design.

The following function resamples rows with probabilities proportional to weights.

In [ ]:
def ResampleRowsWeighted(df, column='finalwgt'):
    weights = df[column]
    cdf = thinkstats2.Cdf(dict(weights))
    indices = cdf.Sample(len(weights))
    sample = df.loc[indices]
    return sample

We can use it to estimate the mean birthweight and compute SE and CI.

In [ ]:
iters = 100
estimates = [ResampleRowsWeighted(live).totalwgt_lb.mean()
             for _ in range(iters)]
Summarize(estimates)

And here's what the same calculation looks like if we ignore the weights.

In [ ]:
estimates = [thinkstats2.ResampleRows(live).totalwgt_lb.mean()
             for _ in range(iters)]
Summarize(estimates)

The difference is non-negligible, which suggests that there are differences in birth weight between the strata in the survey.

# Exercises

**Exercise:** Using the data from the BRFSS, compute the linear least squares fit for log(weight) versus height. How would you best present the estimated parameters for a model like this where one of the variables is log-transformed? If you were trying to guess someone’s weight, how much would it help to know their height?

Like the NSFG, the BRFSS oversamples some groups and provides a sampling weight for each respondent. In the BRFSS data, the variable name for these weights is totalwt. Use resampling, with and without weights, to estimate the mean height of respondents in the BRFSS, the standard error of the mean, and a 90% confidence interval. How much does correct weighting affect the estimates?

Read the BRFSS data and extract heights and log weights.

In [ ]:
import brfss

df = brfss.ReadBrfss(nrows=None)
df = df.dropna(subset=['htm3', 'wtkg2'])
heights, weights = df.htm3, df.wtkg2
log_weights = np.log10(weights)

Estimate intercept and slope.

In [ ]:
# We need to build the y intercept and the slope based on logarithmic weights.
# Thinkstats2.LeastSquarers returns first the y intercept then returns the slope.
inter, slope = thinkstats2.LeastSquares(heights, log_weights)
print("Our y-intercept: {}".format(inter))

# Interesting so we are seeing a much smaller gradual slope here. That is because the log function smoothes it out.
print("the slope m: {}".format(slope))

# Solution goes here
# So our line of best fit starts at about 1 and then increases by .005 for each unit of height. (Based on a logarithmic scale)

Make a scatter plot of the data and show the fitted line.

In [ ]:
# This one is different from the one below.
# The plot below this undoes the log function to give back the values.
# That is why it is called the inverse.
# Log functions can help us normalize the data when it is skewed. (In this case that was the goal)
# This is why we are now getting a straight line.
# The data has been altered by the log function.
# Bet if we plotted the histogram it would have been skewed and now we can make predictions as if the data was normal.
# But we need to be able to get back to the original results so we can talk about them.
# People have trouble thinking in terms of powers.
thinkplot.Scatter(heights, log_weights, alpha=0.01, s=5)
fxs, fys = thinkstats2.FitLine(heights, inter, slope)
thinkplot.Plot(fxs, fys, color='red')
thinkplot.Config(xlabel='Height (cm)', ylabel='log10 weight (kg)', legend=False)


# Solution goes here

Make the same plot but apply the inverse transform to show weights on a linear (not log) scale.

In [ ]:
# Now we are looking back at our heights and weights (The original ones from the dataframe.)
# Alpha makes the chart see through I think.
# I need to figure out what s means. This could be the sample standard deviation but its a bit odd to be exactly 5.
# Usually we have a setup for the plots saying how many items we are going to put on our graph.
thinkplot.Scatter(heights, weights, alpha=0.01, s=5)
fxs, fys = thinkstats2.FitLine(heights, inter, slope)
# this part is important remember fys was built using the log function.
# Logs are returning the power that is why we are plotting by raising to the power of y.
# Raising 10 to the power of fys is the inverse since fys is the log values. So i assume this puts it in base 10 values.
# Our nomral number system.
# oh cool so before we made it linear and normalized. This gave us our line.
# when we are using it to make predicions we put it back into a base 10 system.
# That is why it now curves to look like a exponential graph.
thinkplot.Plot(fxs, 10**fys, color='red')
thinkplot.Config(xlabel='Height (cm)', ylabel='Weight (kg)', legend=False)




# Solution goes here

Plot percentiles of the residuals.

In [ ]:
# again we go back to looking at residuals.
# what we are trying to do here is see what the difference vertically our values differ from the line we created and our observed values.
res = thinkstats2.Residuals(heights, log_weights, inter, slope)
df['residual'] = res

# our setup we want to include the extents of the graph so 130 is the min and 210 is the max x values.
# We also define the increments our graph goes up by.
bins = np.arange(130, 210, 5)

# This function sorts our data into bins based on their index.
indices = np.digitize(df.htm3, bins)
groups = df.groupby(indices)

# This groups up our means from our various samples.
means = [group.htm3.mean() for i, group in groups][1:-1]

# builds our cdf values based on the groups we have
cdfs = [thinkstats2.Cdf(group.residual) for i, group in groups][1:-1]

# We will have three items to print on this graph.
# We want to look at 75 50 25. I think that is because those values translate to
# Quartile Q1 is 25% of our data, Q2 is the middle mean 50% of our data,
# Quartile Q3 is 75% of our data. We want to see if the weighted residuals follow a linear path.
# We are looking for straight parallel lines if they are straight then the differences between
# observed and actual values are linear if they are parallel that means that the varience is the same
# between the different quantiles of data.
thinkplot.PrePlot(3)
for percent in [75, 50, 25]:
    ys = [cdf.Percentile(percent) for cdf in cdfs]
    label = '%dth' % percent
    thinkplot.Plot(means, ys, label=label)

thinkplot.Config(xlabel='height (cm)', ylabel='residual weight (kg)', legend=False)

# So we are seeing that overall our data is not linear.
# The data is pretty linear within a certain range of heights.
# I would say it starts being linear for the top two around about 145 cm then they all follow eachother at about 150 cm.
# This trend goes until about 190 cm.
# So I would feel pretty comfortable making interpolated predictions within the range of 150 cm to 190.
# Also the lines seem to remain parallel within that range with the bottom line being a bit more hard to predict.
# Solution goes here

Compute correlation.

In [ ]:
# I believe rho is that curly p value I was talking to Joseph about on the forums.
# Can you check to make sure I was explaining it correctly to him?
rho = thinkstats2.Corr(heights, log_weights)
rho
# Solution goes here

Compute coefficient of determination.

In [ ]:
# rho and r squared should be the same if everything worked as planned.
# Best of all I believe that rho represents the population.
# This is important because we rarely have info on the population data.
# If all assumptions have been followed that means that given these values we can start to make predictions.
r2 = thinkstats2.CoefDetermination(log_weights, res)
r2
# Solution goes here

Confirm that $R^2 = \rho^2$.


In [ ]:
# my expectation is that this will be very close to zero.
# So it is zero which means we now have information about the variance of the population.
# I feel that the reason everthing works out so nicely here is because we took several samples and found the
# y intercepts and slopes by averaging the results within each of the samples we took.
# Now we are confident that we have a representative sample.
# I am however concerned that r squared is only at 28% so that means that other factors are involved
# when calculating the babys weight. The height only explains 28% of the variance.
rho**2 - r2
# Solution goes here

Compute Std(ys), which is the RMSE of predictions that don't use height.

In [ ]:
# So now we need everything to be in the same unit so we standardize them.
# I think this means we are dividing by standard deviation so they become standard deviation units.
std_ys = thinkstats2.Std(log_weights)
std_ys
# Solution goes here

Compute Std(res), the RMSE of predictions that do use height.

In [ ]:
# We want the root mean square error.
# our goal is to find out how concentrated the values are around our line.
# Lower numbers means the values are more concentrated around the line.
# We saw in the graph before that there was some distance between observed values and the line.
# So I am expecting less than perfect results.
std_res = thinkstats2.Std(res)
std_res
# Cool this was actually quite a bit lower than I expected.
# Low is good it means that our data is concentrated pretty well around the line we created.
# Solution goes here

How much does height information reduce RMSE?

In [ ]:
# hmm this is interesting we are doing more than meets the eye on this formula.
# This will be a probability value between 0 and 1 where we are looking at the values above the point we are interested in.
# Also I think we are actually removing the standardization here.
# Remember we created std_res and std_ys by dividing by standard deviation.
# I think this means we are divding two fractions both of wich have the standard deviation variable as its denominator.
# One copy dot flip later those values would cancel themselves out. This means its litterally impossible to get a value
# out side of the range 0 and 1 for this formula.

1- std_res/std_ys

# Solution goes here

Use resampling to compute sampling distributions for inter and slope.

In [ ]:
# Here we go again we are going to take more samples
t = []
for _ in range(100):
    sample = thinkstats2.ResampleRows(df)
    estimates = thinkstats2.LeastSquares(sample.htm3, np.log10(sample.wtkg2))
    t.append(estimates)

# interesting I did not know what the zip function did.
# apparently it groups items as tuples. I think this is so we can compare our results from before.
# So its appending the new samples to the old ones found in inters and slopes.
# I bet this means we will be graphing the differences to see how they stack up.
inters, slopes = zip(*t)
# Solution goes here

Plot the sampling distribution of slope.

In [ ]:
# It looks a little bit like a sigmoid function. Wish it was a bit smoother. Maybe its because the confidence interval is so small.
# Otherwise it looks pretty good.
cdf = thinkstats2.Cdf(slopes)
thinkplot.Cdf(cdf)
# Solution goes here

Compute the p-value of the slope.

In [ ]:
# My IDE has been frozen for this entire assignment so I hope my predictions are correct.
# I suspect we will get a p-value showing significant results here. Something close to zero.
pvalue = cdf[0]
pvalue
# Solution goes here

Compute the 90% confidence interval of slope.

In [ ]:
# I almost made a mistake here in logic. Similar to the mistake where people look for Percentile(90) thinking that they
# have captured 90% of the data. But the data that I am looking for starts at the 95% mark and omits the bottom 5% capturing 90% of the data between those points.
ci = cdf.Percentile(5), cdf.Percentile(95)
ci
# That is a much tighter range than I expected. Thats kind of insane so now we have a very strong indicator that 90%
# of the time we would have captured our slope. So this means we are very confident that we have chosen a good value for our slope.
# If we choose the average it should fall within this confidence interval.
# Solution goes here

Compute the mean of the sampling distribution.

In [ ]:
mean = thinkstats2.Mean(slopes)
mean
# nice as expected our confidence interval was at 0.0052662 and .0053017
# Lower confidence 0.0052662 our mean is between at .005283 with our upper confidence at .0053017
# Bam we captured it.
# Solution goes here

Compute the standard deviation of the sampling distribution, which is the standard error.

In [ ]:
# I am expecting this to be pretty small because it would be contained within the confidence intervals we discovered before.
# 1 standard deviation from the mean would hold about 68% of the values two standard deviations would hold about 95% three are 99.7% I think.
stderr = thinkstats2.Std(slopes)
stderr
# as expected a very small value. Woot! I am getting this. :)
# Solution goes here

Resample rows without weights, compute mean height, and summarize results.

In [ ]:
# Now we are about to do hypothesis testing. We are trying to assume that the mean represented in our origional results
# is in fact true and then by contradiction find that it is in fact different.
estimates_unweighted = [thinkstats2.ResampleRows(df).htm3.mean() for _ in range(100)]
Summarize(estimates_unweighted)
# Solution goes here

Resample rows with weights.  Note that the weight column in this dataset is called `finalwt`.

In [ ]:
estimates_weighted = [ResampleRowsWeighted(df, 'finalwt').htm3.mean() for _ in range(100)]
Summarize(estimates_weighted)
# Solution goes here
# Awesome in both cases we have captured the mean within the CI
# I also am showing that the original CI range is outside of the weighted CI range.
# Which means that the mean lives in a different location then we had origionaly thought.
# This feels like there is sufficient evidence to reject the null hypothesis in favor of the calculated one based on the CI.
# We had a pvalue to support this idea.
# I am concerned about r squared though it only showed that the variation between our independent and dependant variables at around 28%.
# My IDE keeps freezing so I cant see the value for r-squared but I remember it being too low for my comfort level.
# Im going to go out on a ledge here and say something contrary to the answer key. (I hope this has its own merits)
# The answer key seems to hone in on the fact that there is a difference in means which is supported by what we found with our CIs not over lapping.
# I am however too concerned over the r squared value to be able to pull the trigger on the idea that we have sufficient evidence to reject the null.
# So most likely I have committed a type 2 error in failing to reject the null hypothesis when in fact I should have.
# What do you think? We have strong evidence but can we actually go against what the r squared is telling us?